# Getting Started
 - Shows collision detection
 - Provides a plotting function
 - Creates a slightly optimized submission

In [1]:
import random
from solver.solver import solve_all
from submission.submission import make_submission_df

RNG = random.Random(42)
OUTPUT_FILE = "sample_submission.csv"

tree_data = solve_all(rng=RNG)
df = make_submission_df(tree_data)
df.to_csv(OUTPUT_FILE)